In [1]:
import os
import sys

s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/mouse"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/mouse/nowmm10"

## CFNCluster name
your_cluster_name = "mustafa5"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## Project information
project_name = "mouse_test_proj"

#options: homer
workflow = "homer"

genome = "mm10"

style = "factor"

## If delete cfncluster after job is done.
delete_cfncluster= False

print("variables set")

variables set


In [2]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa5 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="52.43.127.225"
Output:"MasterPrivateIP"="172.31.27.132"
Output:"GangliaPublicURL"="http://52.43.127.225/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.27.132/ganglia/"

connecting
connected


In [3]:
from util import PipelineManager
from util import DesignFileLoader

##can be fastqc, bowite, make_tag_directory, make_UCSC_file, find_peaks, annotate_peaks, pos2bed, find_motifs_genome
analysis_steps = {
                    "fastqc"
                    ,"bowtie"
                    ,"make_tag_directory"
                    ,"make_UCSC_file"
                    #"find_peaks"
                    #,"annotate_peaks"
                    #"pos2bed"
                    #"find_motifs_genome"
                }

design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/test_chip_mouse.txt"


sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("ChiPSeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, style, pairs_list)

[['ERR022267.fastq.gz'], ['ERR022268.fastq.gz'], ['ERR022269.fastq.gz']]
['groupA', 'groupA', 'groupA']
{}
making the yaml file...
copying yaml file to remote master node...
mouse_test_proj.yaml
/shared/workspace/Pipelines/yaml_files/ChiPSeq/homer
executing pipeline...
Executing qsub -V -o /dev/null -e /dev/null /shared/workspace/Pipelines/scripts/run.sh /shared/workspace/Pipelines/yaml_files/ChiPSeq/homer/mouse_test_proj.yaml /shared/workspace/logs/ChiPSeq/homer/mouse_test_proj ChiPSeq_homer
